<a href="https://colab.research.google.com/github/Kaisano/GCollab_torrentDL/blob/main/torrent_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [1]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!pip install libtorrent
!apt install 7zip
!pip install dropbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.0 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 60.7 MB/s  0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  7zip

# Imports
rich.progress for concurrent progress tracking<br>
libtorrent as torrent client<br>
dropbox as storage solution<br>

In [37]:
import sys
import os, subprocess, glob
import time
from timeit import default_timer as timer

from rich.progress import Progress, TextColumn, BarColumn, TaskProgressColumn, TimeRemainingColumn

from pathlib import Path
import libtorrent as lt

import dropbox
from dropbox.files import WriteMode
from dropbox.exceptions import ApiError, AuthError

from concurrent.futures import ThreadPoolExecutor, Future

# Setup Filesystem

In [7]:
magnets_path = Path("/content/magnet.links")
torrents_path = Path("/content/Torrent Files")
tkn_path = Path("/content/dbx.tkn")

if not torrents_path.exists():  torrents_path.mkdir()
if not magnets_path.exists():   magnets_path.touch()
if not tkn_path.exists():       tkn_path.touch()

In [60]:
# if path.stat().st_size == 0:

links = list(magnets_path.read_text(encoding="utf-8").splitlines())
TOKEN = str(Path("dbx.tkn").read_text().rstrip().strip())

In [71]:
def get_metadata(progress, task, handle: lt.torrent_handle, timeout=90):
  progress.update(task, filename="Obtaining Metadata...")
  status = handle.status()

  refresh_time_ms = 0.8
  start_time = timer()
  while not status.has_metadata:
    time.sleep(refresh_time_ms/1000)
    status = handle.status()

    if start_time - timer() > timeout:
      # print("Timed out on obtaining metadata")
      return None
  progress.update(task, filename=handle.status().name)

  return status

# TODO: Handle torrent files
def download_torrent(progress: Progress, link):
  save_path = Path("/content/Torrents/")
  if not save_path.exists(): save_path.mkdir()

  ses = lt.session()

  try:
    atp = lt.parse_magnet_uri(link)           # atp = add torrent parameters
  except Exception as e:
    # print("Failed to parse magnet URI: ", e)
    return None
  atp.save_path = str(save_path.as_posix())

  task = progress.add_task(
        "download",
        total=100,
        filename="Initialized client",
        speed="0.0 kB/s",
        )

  handle = ses.add_torrent(atp)
  status = get_metadata(progress, task, handle)

  if status is None:
    return None

  # while not status.is_seeding:
  #   status = handle.status()

  #   progress.update(task,
  #               completed=status.progress * 100,
  #               speed=f"{status.download_rate/1000:.1f} kB/s",
  #           )
  # progress.update(task,
  #               completed=status.progress * 100,
  #               speed=f"{status.download_rate/1000:.1f} kB/s",
  #           )

  return Path("/content/Torrents/" + handle.status().name)


In [72]:
def process_magnet_links(progress, links):
  def tor_factory(link):
    return download_torrent(progress, link)

  completed_torrents = {}
  with ThreadPoolExecutor() as executor:
    for link, file in zip(links, executor.map(tor_factory, links)):
      completed_torrents[file.name] = (file)

  return completed_torrents

torrents = None
with Progress(
  TextColumn("[bold]{task.fields[filename]}"),
  BarColumn(),
  TaskProgressColumn(),
  TimeRemainingColumn(),
  TextColumn("{task.fields[speed]}"),
  ) \
as progress:
  torrents = process_magnet_links(progress, links)

Output()

In [73]:
print(torrents)

{'[Erai-raws] City the Animation - 01 [1080p AMZN WEBRip HEVC EAC3][MultiSub][67052423].mkv': PosixPath('/content/Torrents/[Erai-raws] City the Animation - 01 [1080p AMZN WEBRip HEVC EAC3][MultiSub][67052423].mkv'), '[Erai-raws] City the Animation - 05 [1080p AMZN WEB-DL AVC EAC3][MultiSub][C9E5B1DE].mkv': PosixPath('/content/Torrents/[Erai-raws] City the Animation - 05 [1080p AMZN WEB-DL AVC EAC3][MultiSub][C9E5B1DE].mkv'), 'Yotsuba&! (Digital) (danke-Empire)': PosixPath('/content/Torrents/Yotsuba&! (Digital) (danke-Empire)')}


In [ ]:
# subprocess.run(["7za", "a", "-v1024m", "/content/Torrent.zip", "/content/Torrent/"])

CompletedProcess(args=['7za', 'a', '-v1024m', '/content/Torrent.zip', '/content/Torrent/'], returncode=2)

In [ ]:
# def upload(
#     access_token,
#     file_path,
#     target_path,
#     timeout=900,
#     chunk_size=4 * 1024 * 1024,
# ):
#     dbx = dropbox.Dropbox(access_token, timeout=timeout)
#     with open(file_path, "rb") as f:
#         file_size = os.path.getsize(file_path)
#         if file_size <= chunk_size:
#             print(dbx.files_upload(f.read(), target_path))
#         else:
#             with tqdm(total=file_size, desc="Uploaded") as pbar:
#                 upload_session_start_result = dbx.files_upload_session_start(
#                     f.read(chunk_size)
#                 )
#                 pbar.update(chunk_size)
#                 cursor = dropbox.files.UploadSessionCursor(
#                     session_id=upload_session_start_result.session_id,
#                     offset=f.tell(),
#                 )
#                 commit = dropbox.files.CommitInfo(path=target_path)
#                 while f.tell() < file_size:
#                     if (file_size - f.tell()) <= chunk_size:
#                         print(
#                             dbx.files_upload_session_finish(
#                                 f.read(chunk_size), cursor, commit
#                             )
#                         )
#                     else:
#                         dbx.files_upload_session_append(
#                             f.read(chunk_size),
#                             cursor.session_id,
#                             cursor.offset,
#                         )
#                         cursor.offset = f.tell()
#                     pbar.update(chunk_size)

# torrents = list(Path("/content/").glob("Torrent.zip.*"))
# # upload(TOKEN, torrents[1].as_posix(), "/Torrent/" + torrents[1].name)
# # for torrent in torrents:


# # run uploads in parallel
# with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
#     futures = [
#         executor.submit(upload, TOKEN, torrent.as_posix(), "/Torrent/" + torrent.name)
#         for torrent in torrents
#     ]
#     for future in concurrent.futures.as_completed(futures):
#         try:
#             future.result()
#         except Exception as e:
#             print("Upload failed:", e)

Uploaded: 931135488it [03:34, 4341456.09it/s]

FileMetadata(client_modified=datetime.datetime(2025, 10, 6, 6, 35, 57), content_hash='7da9e45bf034cf26c2e008db41876fc4150e4e0eef238532d2e7ea92cc9631cd', export_info=NOT_SET, file_lock_info=NOT_SET, has_explicit_shared_members=NOT_SET, id='id:uarvdFOTHrAAAAAAAAAAHg', is_downloadable=True, media_info=NOT_SET, name='Torrent.zip.001', parent_shared_folder_id=NOT_SET, path_display='/Torrent/Torrent.zip.001', path_lower='/torrent/torrent.zip.001', preview_url=NOT_SET, property_groups=NOT_SET, rev='0164077aa05e6c60000000300d37ac1', server_modified=datetime.datetime(2025, 10, 6, 6, 35, 58), sharing_info=NOT_SET, size=928419183, symlink_info=NOT_SET)
